In [49]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import csv
import numpy as np
import pandas as pd
import re
from auto_login import get_driver_login

In [50]:
def people_scrape(search_term, num_pages):  
    loc_list = []
    url_list = []
    headline_list = []
    name_list = []
    current_job_list = []
#                 function to automate search bar, search focus
#                 from your linkedin homepage, collecting data from
#                 search including name, location(secondary_deets), 
#                 headline(primary_deets) from condensed profiles returned from
#                 search results. requires string entry for search term
#                 and int input for number of pages (num_pages) 
#                 that you wish to scrape from results.

    # activate search bar cursor with click
    driver.find_element_by_css_selector("div#global-nav-search ").click()
    time.sleep(2)
    # send keyboard entry "div[id='oc-background-section']")for search terms
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(search_term)
    # send enter key to activate search
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(Keys.RETURN)
    # wait for results to load
    driver.implicitly_wait(6)
    #w.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.reusable-search__entity-results-list')))
    element = driver.find_element_by_css_selector('ul.reusable-search__entity-results-list ')
    # scroll to element containing target(people_banner)  allowing ajax elements to load
    driver.execute_script("arguments[0].scrollIntoView();", element)
    driver.implicitly_wait(5)
   # locate banner under search type results
    people_banner = driver.find_element_by_link_text(f"See all people results")
    # js function to click banner/button to see additional results under jobs, people, or posts                      
    driver.execute_script('arguments[0].click();',people_banner)
    # pause to allow page to load
    driver.implicitly_wait(6)
    # starting with pagination page 1 to increase as pagination occurs                     
    page_number = 1

    # set while loop to define pagination and data collection conditions
    while page_number <= num_pages:
        print("Processing page: " + str(page_number))
    # find all results on page
        links = driver.find_elements_by_css_selector("div.entity-result__content ")
   # pause for page load
        time.sleep(2)
    # iterate through results
        for l in links:
    # retrieve profile url
            url = l.find_element_by_css_selector("span.entity-result__title a.app-aware-link")
    # add to urls list
            url_list.append(str(url.get_attribute("pathname")))
     # locating elements containing text needed
            details = l.find_elements_by_css_selector("div.linked-area")
    # the first element has the first three lines of text in the container
            deets = details[0]  
    # split text to assign elements appropriately
            text = deets.get_attribute('innerText').split('\n')
    # retrieve name/add to list
            name_list.append(text[0])
    # retrieve location/add to list   
            loc_list.append(text[-1])
    # retrieve headline/add to list
            headline_list.append(text[-2])
    # the second element selected contains the 'Current:' job text 
            try:
                current_job = details[1]
    # removing the 'Current:' string from text
                current_job_list.append(current_job.get_attribute('innerText').split(':')[1])
            except (NoSuchElementException, IndexError):
                current_job_list.append('nan')
        time.sleep(3)
        page_number+=1
    # navigate using pagination function
        if page_number < num_pages:
            goto_next_page()
    # print to verify page during processing
            print(f"attempting to navigate to search results page {page_number}")
            time.sleep(5)

   
    # create dataframe with extracted information and save as csv file
    df = pd.DataFrame()                      
    df['name'] = name_list
    df['url'] = url_list
    df['current_job'] = current_job_list
    df['location'] = loc_list
    df['headline'] = headline_list
    # add complete url information for use in complete profile scraping
    for row in df:
        df['fetch'] = 'https://www.linkedin.com' + df.url + '/'
    df.to_csv(f'{search_term}.csv')
    # verify save
    print(f'{search_term}.csv saved')
    return df

In [72]:
def people_scrape_by_start_page(search_term, start_page, num_pages):  
    loc_list = []
    url_list = []
    headline_list = []
    name_list = []
    current_job_list = []
#                 function to continue data scraping at page number
#                 from your linkedin search, collecting data from
#                 search including name, location(secondary_deets), 
#                 headline(primary_deets) from condensed profiles returned from
#                 search results. requires string entry for search term
#                 and int input for number of pages (num_pages) 
#                 that you wish to scrape from results.
#                 start_page argument must be in integer format displayed in pagination bar
#                 in active search


    # starting with pagination page 1 to increase as pagination occurs                     
    goto_page(start_page)
    page_number = 1
    
    # set while loop to define pagination and data collection conditions
    while page_number <= num_pages:
        print("Processing page: " + str(start_page))
    # find all results on page
        links = driver.find_elements_by_css_selector("div.entity-result__content ")
   # pause for page load
        time.sleep(2)
    # iterate through results
        for l in links:
    # retrieve profile url
            url = l.find_element_by_css_selector("span.entity-result__title a.app-aware-link")
    # add to urls list
            url_list.append(str(url.get_attribute("pathname")))
     # locating elements containing text needed
            details = l.find_elements_by_css_selector("div.linked-area")
    # the first element has the first three lines of text in the container
            deets = details[0]  
    # split text to assign elements appropriately
            text = deets.get_attribute('innerText').split('\n')
    # retrieve name/add to list
            name_list.append(text[0])
    # retrieve location/add to list   
            loc_list.append(text[-1])
    # retrieve headline/add to list
            headline_list.append(text[-2])
    # the second element selected contains the 'Current:' job text 
            try:
                current_job = details[1]
    # removing the 'Current:' string from text
                current_job_list.append(current_job.get_attribute('innerText').split(':')[1])
            except (NoSuchElementException, IndexError):
                current_job_list.append('nan')
        time.sleep(3)
        page_number+=1
        start_page+=1
    # navigate using pagination function
        if page_number < num_pages:
            goto_next_page()
    # print to verify page during processing
            print(f"attempting to navigate to search results page {start_page}")
            time.sleep(5)

   
    # create dataframe with extracted information and save as csv file
    df = pd.DataFrame()                      
    df['name'] = name_list
    df['url'] = url_list
    df['current_job'] = current_job_list
    df['location'] = loc_list
    df['headline'] = headline_list
    # add complete url information for use in complete profile scraping
    for row in df:
        df['fetch'] = 'https://www.linkedin.com' + df.url + '/'
    df.to_csv(f'{search_term}.csv')
    # verify save
    print(f'{search_term}.csv saved')
    return df

In [52]:
#function to locate and interact with "next" button at bottom of search
def goto_next_page():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    w = WebDriverWait(driver, 15)
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.artdeco-pagination.ember-view")))
    go_to_next = driver.find_element_by_css_selector('button[aria-label="Next"]')
    driver.execute_script('arguments[0].click();',go_to_next)
    
#function to locate and interact with "next" button at bottom of search
def goto_page(page_number):
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    w = WebDriverWait(driver, 15)
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.artdeco-pagination.ember-view")))
    pages = driver.find_element_by_css_selector('ul.artdeco-pagination__pages')
    page_button = f'Page {page_number}'
    target_page = driver.find_element_by_css_selector(f'button[aria-label="{page_button}"]')
    driver.execute_script('arguments[0].click();',target_page)


In [53]:
def get_experience():
    time.sleep(3)
    w = WebDriverWait(driver, 10)
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.profile-detail")))
    h2_items = driver.find_elements_by_tag_name(
        "h2")
    driver.execute_script("arguments[0].scrollIntoView();", h2_items[1])
    background = driver.find_element_by_css_selector(
        "div#oc-background-section")
    driver.execute_script("arguments[0].scrollIntoView();", background)
    time.sleep(5)
    # locate experience section element
    exp = background.find_element_by_css_selector(
                "section#experience-section.pv-profile-section.experience-section.ember-view")
    # access individual job containers in list format 
    history = exp.find_elements_by_css_selector('li.pv-entity__position-group-pager')
    job_count = len(history)
    details = history[0]
#          use of  try/except clause to locate element to avoid 'element not found' error which halts program
# job title
    try:
        job = details.find_element_by_tag_name(
                        'h3').get_attribute('innerText')
    except NoSuchElementException:
        job = 'nan'
 # company of employment
    try:
        company = details.find_element_by_tag_name(
                        'p.pv-entity__secondary-title').get_attribute('innerText')
        company = re.sub('Full-time', '', company)
    except NoSuchElementException:
        company = 'nan'
# location of employment
    try:
        location = details.find_element_by_css_selector(
                        'h4.pv-entity__location').get_attribute('innerText')
        location = location.split('\n', 1)[1]
    except NoSuchElementException:
        location = 'nan'
# dates of employment
    try:
        date = details.find_element_by_css_selector(
                        "h4.pv-entity__date-range").get_attribute('innerText').split(' ', 2)[-1]
    except NoSuchElementException:
        date = 'nan'
        
    return job_count, job, company, location, date
    

In [54]:
def get_email():
#     function accessing and retrieves email and name from profile header, extracts first name 
    # scroll to top of profile where email is located
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    header = driver.find_elements_by_css_selector(
        "ul.pv-top-card--list")
    # name on profile
    name = header[0].get_attribute('innerText')
   # first name derived from name
    fname = name.split(' ')[0]
    # find element with email contained within
    contact_info = driver.find_element_by_css_selector(
        'a[data-control-name="contact_see_more"]')
     # js function to click banner/button to see additional results under jobs, people, or posts                      
    driver.execute_script('arguments[0].click();',contact_info)
    time.sleep(3)
    # not everyone provides email so try/except block used
    try:
        container =  driver.find_elements_by_css_selector(
            'div.pv-contact-info__ci-container')
        email = container[1].get_attribute('innerText')
    except (NoSuchElementException, IndexError):
        email = 'nan'
# close pop up with contact info
    close_popup = driver.find_element_by_css_selector(
            'button[aria-label="Dismiss" ]')
    close_popup.click()
    return name, fname, email

In [55]:
def profile_connect(message):
    # scroll to top of profile to ensure elements can be found by webdriver
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(5)
    # locate & click 'Connect' button located at top of profile
    try:
        driver.find_element_by_class_name('pv-s-profile-actions--connect').click()
        action_bar = driver.find_element_by_css_selector("div.artdeco-modal__actionbar")
    # locate 'Add a note' button by class
        action_bar.find_element_by_class_name('mr1').click()
        time.sleep(3)
    # action to send text from message to input box
        message_input = driver.find_element_by_id('custom-message').send_keys(message)
        # send the message & connection request
        driver.find_element_by_class_name('ml1').click()
        
    except (NoSuchElementException, ElementNotInteractableException):
        pass


In [56]:
def profile_scrape(list_profile_urls):
    job_count_list = []
    job_list = []
    company_list = []
    date_list = []
    location_list = []
    names_list = []    
    email_list = []
    fname_list = []
    for url in list_profile_urls:
        driver.get(url)
        job_count, job, company, location, date = get_experience()
        job_count_list.append(job_count)
        job_list.append(job)
        company_list.append(company)
        date_list.append(date)
        location_list.append(location)
        name, fname, email = get_email()
        names_list.append(name)
        fname_list.append(fname)
        email_list.append(email)
    df = pd.DataFrame()
    df['profile_name'] = names_list
    df['job1'] = job_list
    df['job_count'] = job_count_list
    df['company1'] = company_list
    df['location1'] = location_list
    df['dates1'] = date_list
    df['first_name'] = fname_list
    df['email'] = email_list
    return df

def make_connection(url_message_dict):
    counter = 1

    for key, value in url_message_dict.items():
        url = key
        message = value
        driver.get(url)
        profile_connect(message)
        print('connection to :  ' +url+ ' is complete')
        counter+=1
    print('Process complete '+str(counter)+ ' connections requested')

In [57]:
def search_scrape_connect(search_term, num_pages):   

    search_df = people_scrape(search_term, num_pages)
    
    profile_urls = search_df['fetch']
    
    detail_df = profile_scrape(profile_urls)

    df = pd.concat([search_df, detail_df], axis=1)
    df = df.loc[df.company1 != 'nan']
    df = df.loc[df.job1 != 'nan']

    df['personalized_message'] = ("Hi " + df.first_name +", I am a data scientist in the DC area. My background is in video editing. "
                                                       "After completing the Flatiron data science program,  I am transitioning into the DS/ML career field" 
                                                       ", hoping to segue into AI."
                                                       " I see that you are a " + df.job1 + "at "+ df.company1+
                                                       ", so I just wanted to reach out, connect, and say hello!")  

    url_message_dict = dict(zip(list(df.fetch), list(df.personalized_message)))
    df.to_csv(f'{search_term}.csv')
    print(f'initial {search_term}.csv overwritten, final csv file saved')
    make_connection(url_message_dict)
    return df
    driver.quit()

In [58]:
def scrape_connect(search_df):   
    profile_urls = search_df.fetch
    detail_df = profile_scrape(profile_urls)

    df = pd.concat([search_df, detail_df], axis=1)
    df = df.loc[df.company1 != 'nan']
    df = df.loc[df.job1 != 'nan']

    df['personalized_message'] = ("Hi " + df.first_name +", I am a data scientist in the DC area. My background is in video editing. "
                                                       "After completing the Flatiron data science program,  I am transitioning into the DS/ML career field" 
                                                       ", hoping to segue into AI."
                                                       " I see that you are a " + df.job1 + "at "+ df.company1+
                                                       ", so I just wanted to reach out, connect, and say hello!")  

    url_message_dict = dict(zip(list(df.fetch), list(df.personalized_message)))
    df.to_csv('connections.csv')
    print(f'initial {search_term}.csv overwritten, final csv file saved')
    make_connection(url_message_dict)
    return df
    driver.quit()

In [59]:
# utilized in connect function to obtain name, job and company info for message without saving as csv
def get_job_company():
    time.sleep(3)
    w = WebDriverWait(driver, 10)
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.profile-detail")))
    h2_items = driver.find_elements_by_tag_name(
        "h2")
    driver.execute_script("arguments[0].scrollIntoView();", h2_items[1])
    background = driver.find_element_by_css_selector(
        "div#oc-background-section")
    driver.execute_script("arguments[0].scrollIntoView();", background)
    time.sleep(6)
    # locate experience section element
    exp = background.find_element_by_css_selector(
                "section#experience-section.pv-profile-section.experience-section.ember-view")
    # access individual job containers in list format 
    history = exp.find_elements_by_css_selector('li.pv-entity__position-group-pager')
    job_count = len(history)
    details = history[0]
#          use of  try/except clause to locate element to avoid 'element not found' error which halts program
# job title
    try:
        job = details.find_element_by_tag_name(
                        'h3').get_attribute('innerText')
    except NoSuchElementException:
        job = 'nan'
 # company of employment
    try:
        company = details.find_element_by_tag_name(
                        'p.pv-entity__secondary-title').get_attribute('innerText')
        company = re.sub('Full-time', '', company)
    except NoSuchElementException:
        company = 'nan'
    
    job_info = [job, company]
    return job_info

In [60]:
def get_first_name():
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    header = driver.find_elements_by_css_selector(
            "ul.pv-top-card--list")
    # name on profile
    header_text = header[0].get_attribute('innerText')
   # first name derived from name
    fname = header_text.split(' ')[0]
    return fname

In [61]:
def connect_name(search_df):   
# use to complete connection process when timeout errors occur mid process for scrape part of function
#  use df obtained from search function as search_df 
# use this if df names are incomplete or if only urls are present, this function includes the get_first_name function to fill message
    profile_urls = search_df.fetch
    for url in profile_urls:
        driver.get(url)
        fname = get_first_name()
        job_info = get_job_company()
        current_job = job_info[0]
        current_company = job_info[1]
        
        if current_job != 'nan':
            if current_company != 'nan':
                message = ("Hi " + fname +", I am a data scientist in the DC area. My background is in video editing. ",
                                                               "I am transitioning into the DS/ML career field, hoping to segue into AI.", 
                                                               " How do you like your role as " + current_job + " at "+ current_company+"?",
                                                               "Anyway, I just wanted to reach out, connect, and say hello!")  


                profile_connect(message)
                print(f'Connection complete: {fname} // {current_company} // {current_job} ')

In [62]:
def connect(search_df):   
# use to complete connection process when timeout errors occur mid process for scrape part of function
#  use df obtained from search function as search_df 
# uses name acquired in people_search to complete message
    url_name_dict = dict(zip(list(search_df.name), list(search_df.fetch)))
    for name, url in url_name_dict.items():
        driver.get(url)
   # first name derived from name
        fname = name.split(' ')[0]

        job_info = get_job_company()
        current_job = job_info[0]
        current_company = job_info[1]
        if current_job != 'nan':
            if current_company != 'nan':
                message = ("Hi " + fname +", I am a data scientist in the DC area. My background is in video editing. ",
                                                               "I am transitioning into the DS/ML career field, hoping to segue into AI.", 
                                                               " How do you like your role as " + current_job + " at "+ current_company+"?",
                                                               "Anyway, I just wanted to reach out, connect, and say hello!")  


                profile_connect(message)
                print(f'Connection complete: {name} // {current_company} // {current_job} ')


In [12]:
driver =  get_driver_login('email@gmail.com', 'password1!', PATH = "C:\Program Files (x86)\chromedriver.exe")
# does not return second csv with more details like email, location, dates, etc but tends to run smoother 
search_df = people_scrape('data scientist', 3)
connect(search_df)
# does return second csv with more details like email, location, dates, etc, connect within function
search_scrape_connect('data scientist', 3)

In [81]:
df = people_scrape('ai engineer', 10)

Processing page: 1
attempting to navigate to search results page 2
Processing page: 2
attempting to navigate to search results page 3
Processing page: 3
attempting to navigate to search results page 4
Processing page: 4
attempting to navigate to search results page 5
Processing page: 5
attempting to navigate to search results page 6
Processing page: 6
attempting to navigate to search results page 7
Processing page: 7
attempting to navigate to search results page 8
Processing page: 8
attempting to navigate to search results page 9
Processing page: 9
Processing page: 10
ai engineer.csv saved


In [79]:
people_scrape_by_start_page('ai engineer2', 11, 20)
people_scrape_by_start_page('ai engineer3', 31, 20)

Processing page: 65
attempting to navigate to search results page 66
Processing page: 66
attempting to navigate to search results page 67
Processing page: 67
attempting to navigate to search results page 68
Processing page: 68
attempting to navigate to search results page 69
Processing page: 69
attempting to navigate to search results page 70
Processing page: 70
attempting to navigate to search results page 71
Processing page: 71
attempting to navigate to search results page 72
Processing page: 72
attempting to navigate to search results page 73
Processing page: 73
attempting to navigate to search results page 74
Processing page: 74
attempting to navigate to search results page 75
Processing page: 75
attempting to navigate to search results page 76
Processing page: 76
attempting to navigate to search results page 77
Processing page: 77
attempting to navigate to search results page 78
Processing page: 78
attempting to navigate to search results page 79
Processing page: 79
attempting to 

,name,url,current_job,location,headline,fetch
0,Carl Baumbach,/in/carlbaumbach,...and Hadoop 2.6.5+ Graduated with a Master’...,Washington DC-Baltimore Area,Data Engineer at AE Strategies,https://www.linkedin.com/in/carlbaumbach/
1,Medha Banda,/in/medha-banda-8b5210159,Data Engineer at TISTA Science and Technology...,"Potomac, MD",Data Engineer| Hadoop| Bigdata,https://www.linkedin.com/in/medha-banda-8b5210...
2,Brian Duffy,/in/bduffy0329,Data Engineer at LMI,Washington DC-Baltimore Area,Senior Data Engineer at LMI,https://www.linkedin.com/in/bduffy0329/
3,Haseeb Ahamed Naseem,/in/haseebahamed,Lead Data Engineer at Capital One,"Falls Church, VA",Lead Data Engineer,https://www.linkedin.com/in/haseebahamed/
4,Kavyashree Anantha Raman,/in/kavyashreeanantharaman,Data Mining Engineer at Hughes Network Systems,Washington DC-Baltimore Area,Data Analytics Engineer,https://www.linkedin.com/in/kavyashreeananthar...
...,...,...,...,...,...,...
169,Annette C.,/in/annettechun,"Software Engineer at Comscore, Inc.",Washington DC-Baltimore Area,Data Engineer at Amazon Web Services (AWS),https://www.linkedin.com/in/annettechun/
170,Sarah El-Sharkawi,/in/sarah-el-sharkawi-335a6a117,Data & Analytics Consultant at EY - Analyzed ...,"Leesburg, VA",Senior Data Engineer at EY,https://www.linkedin.com/in/sarah-el-sharkawi-...
171,David Yu,/in/david-yu-07aa6539,"Data Analyst at Comscore, Inc.",Washington DC-Baltimore Area,Data Engineer at Accenture Federal Services,https://www.linkedin.com/in/david-yu-07aa6539/
172,Nirmalkumar Elumalai,/in/nirmalkumar-elumalai-06b6a9178,Senior Data Engineer at Cognizant,"McLean, VA",Data Engineer at Cognizant,https://www.linkedin.com/in/nirmalkumar-elumal...


In [93]:
def rle_encode(input_string):
    encoded_string = ''
    prev_char = ''
    count = 1
    if not input_string: return ''
    for char in input_string:
        if char != prev_char:
            if prev_char:
                encoded_string += str(count) + prev_char
            count = 1
            prev_char = char
        else:
            count += 1
    else:
        encoded_string += str(count) + prev_char
        return encoded_string

In [95]:
encoded_val = rle_encode('AAAAAAFDDCCklgjfdEEEEEEEEEEEE')
print(encoded_val.encode())

b'6A1F2D2C1k1l1g1j1f1d12E'


In [124]:
def compress(input_string):
    encoded_string = ''
    prev_char = ''
    count = 1

    for char in input_string:
        if char != prev_char:
            if prev_char:
                encoded_string += str(count) + prev_char
                count = 1
                prev_char = char
            else:
                count += 1
        else:
            encoded_string += str(count) + prev_char
    return encoded_string.encode('ascii')

In [136]:


def compress(input_string):
    encoded_string = ''
    prev_char = ''
    count = 1
    if not input_string:
        return ''.encode()
    for char in input_string:
        if char != prev_char:
            if prev_char:
                encoded_string += str(count) + prev_char
                count = 1
                prev_char = char
        else:
            count += 1
    else:
        encoded_string += str(count) + prev_char
        return encoded_string.encode("utf-16")

In [137]:
compress('AAAAAAFDDCCklgjfdEEEEEEEEE EEE')


b'\xff\xfe1\x00'

In [106]:
"a1".encode('utf-8')

b'a1'

In [232]:
def rle_for_loop(str_input):
    encoded_string, prev_char, count = '', '', 1
    if not str_input: 
        return ''.encode()
    for char in str_input:
        if char != prev_char:
            if prev_char:
                encoded_string += str(count) + prev_char
                count=1
            else:
                encoded_string += str(count) + char
        prev_char = char
    return encoded_string

In [ ]:
def rle_while_loop(str_input):
    encoded_string = ""
    j = 0
    while j < len(str_input):
        count = 1
        while j + 1 < len(str_input) & str_input[j] = string_input[j + 1]:
            count += 1
            j += 1
        encoded_string += str(count) + string_input[j]
        j += 1
    return encoded_string
    

In [245]:
from collections import OrderedDict
def rle_counter(str_input):
    encoded_string = ""
    encode_dict = OrderedDict.fromkeys(str_input, 0)
   # iterating over the string
    for char in str_input:
      # incrementing the frequency
      encode_dict[char] += 1
    
    for key, value in encode_dict.items():
        encoded_string += key + str(value)
    return encoded_string

In [246]:
rle_encode_counter('kjkk')

'k3j1'

In [247]:
from itertools import groupby
def rle_lc(str_input):
    grouped = [list(g) for k, g in groupby(str_input)]
    return ''.join(['{}{}'.format(k, sum(1 for _ in g)) for k, g in groupby(str_input)])

In [248]:
rle_encode_lc('kjkk')

'k1j1k2'

In [249]:
def rle_gen(data: str) -> str:
    # returns run length encoded string for data
    return "".join(f"{x}{sum(1 for _ in y)}" for x, y in groupby(data))

In [250]:
rle_encode_gen('kjkk')

'k1j1k2'